<a href="https://colab.research.google.com/github/d1vv1/ppe-detection-pipeline/blob/main/ppe_train_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Access Secrets**
Load secrets into the environmet

In [8]:
from google.colab import userdata

# Load all secrets as environment variables
WANDB_API_KEY = userdata.get('WANDB_API_KEY')
B2_ACCESS_KEY_ID = userdata.get('B2_ACCESS_KEY_ID')
B2_SECRET_ACCESS_KEY = userdata.get('B2_SECRET_ACCESS_KEY')
GITHUB_USER = userdata.get('GITHUB_USER')
GITHUB_EMAIL = userdata.get('GITHUB_EMAIL')
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')

# Configure W&B
import os
os.environ['WANDB_API_KEY'] = WANDB_API_KEY

## Clone repo

In [9]:
# Cell 2: Clone or Update Your Repo

import os
import shutil

REPO_URL = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/ppe-detection-pipeline.git"
REPO_DIR = "ppe-detection-pipeline"

# If the repo folder exists, delete it to ensure a fresh pull
if os.path.exists(REPO_DIR):
    shutil.rmtree(REPO_DIR)

# Clone the latest version
!git clone {REPO_URL}

# Move into the project directory
%cd {REPO_DIR}

# Optional: confirm the latest commit pulled
!git log --oneline -1


Cloning into 'ppe-detection-pipeline'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 72 (delta 24), reused 67 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (72/72), 17.85 KiB | 3.57 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/content/ppe-detection-pipeline/ppe-detection-pipeline
066ce6f (HEAD -> main, origin/main, origin/HEAD) Enhance W&B error handling in training script by introducing alternative initialization methods for authentication issues. Improved logging messages to clarify potential causes and fallback strategies for W&B integration.


## Install dependencies

In [10]:
!pip install "dvc[s3]"
!pip install -r requirements.txt
!pip install -r api/requirements.txt # Installs ultralytics, etc.

## Configure DVC Remote

In [11]:
!dvc remote modify --local myremote access_key_id {B2_ACCESS_KEY_ID}
!dvc remote modify --local myremote secret_access_key {B2_SECRET_ACCESS_KEY}

## Mount G-drive and unzip

In [12]:
from google.colab import drive
drive.mount('/content/drive')

print("Google Drive mounted. Unzipping ppe-dataset.zip...")

!unzip /content/drive/MyDrive/ppe-dataset.zip -d data/

print("Dataset unzipped successfully.")

Streaming output truncated to the last 5000 lines.
  inflating: data/ppe-dataset/train/labels/frame2599_jpg.rf.35c0e10af1e8f4f546290ffdba9f77fc.txt  
  inflating: data/ppe-dataset/train/labels/004911_jpg.rf.42655d4dd11b4b5aaf096706c9b927e3.txt  
  inflating: data/ppe-dataset/train/labels/003873_jpg.rf.8098043422187b030d02e9a70feea26a.txt  
  inflating: data/ppe-dataset/train/labels/002476_jpg.rf.00b13076e297c757efea2b4e4957032c.txt  
  inflating: data/ppe-dataset/train/labels/image_39_jpg.rf.b4276e87d89f18d8a4c30bedf9c11fb1.txt  
  inflating: data/ppe-dataset/train/labels/005509_jpg.rf.2a7465e6946db7db7a2ade814ef6f462.txt  
  inflating: data/ppe-dataset/train/labels/006600_jpg.rf.6de4288d1a095b8786aaeb675dfffda1.txt  
  inflating: data/ppe-dataset/train/labels/people-3778-_jpg.rf.4a07f968b18993019096014a2fe7bf60.txt  
 extracting: data/ppe-dataset/train/labels/1250_jpg.rf.20e42f9f8868015ffc3b94ec58a2af39.txt  
  inflating: data/ppe-dataset/train/labels/people-374-_jpg.rf.7fff57eeeb5fb0

## Run the training

**Options:**
- Set `resume=True` to continue from last checkpoint if training was interrupted
- Set `google_drive_backup` to backup checkpoints to Google Drive (recommended for Colab)
- Checkpoints are saved every epoch by default for Colab safety

In [ ]:
# Option 1: Run with default settings (checkpoints saved every epoch)
# !python scripts/train.py

# Option 2: Run with Google Drive backup (RECOMMENDED for Colab)
# This will backup checkpoints to Google Drive so you can resume even if Colab disconnects
from scripts.train import train_model

train_model(
    model_size='m',
    epochs=25,
    imgsz=640,
    batch=8,
    save_period=1,  # Save checkpoint every epoch (default, good for Colab)
    val=True,  # Enable validation (uses valid/images from data.yaml)
    resume=False,  # Set to True to resume from last checkpoint
    google_drive_backup='/content/drive/MyDrive/ppe-models',  # Backup to Google Drive
)

# If training gets interrupted, resume with:
# train_model(resume=True, google_drive_backup='/content/drive/MyDrive/ppe-models')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Starting PPE Detection Model Training
Working directory: /content/ppe-detection-pipeline
✓ Validation dataset found: 8814 files in /content/ppe-detection-pipeline/data/ppe-dataset/valid/images
✓ Google Drive backup enabled: /content/drive/MyDrive/ppe-models


wandb: Currently logged in as: divyanshumahi4 (divyanshumahi4-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✓ W&B API key found in environment


✓ Weights & Biases initialized: ppe-detection-pipeline/yolov8m-ppe-20251109_111019
  View run at: https://wandb.ai/divyanshumahi4-college/ppe-detection-pipeline/runs/54vd7ask
Loading pre-trained model: yolov8m.pt
✓ Model loaded: YOLOv8m
✓ W&B logging enabled (wandb initialized, Ultralytics will use it automatically)

Training Configuration:
  data: /content/ppe-detection-pipeline/data/ppe-dataset/data.yaml
  epochs: 25
  imgsz: 640
  batch: 8
  name: yolov8m_ppe_detection
  patience: 50
  save_period: 1
  val: True
  plots: True
  verbose: True
  save: True
  save_json: True
  project: runs
  Validation: ENABLED
  Checkpoint saving: Every 1 epoch(s)
  W&B logging: ENABLED (wandb initialized)

Starting model training...
  Metrics will be logged to W&B in real-time during training
  Checkpoints will be saved every epoch for Colab safety

Ultralytics 8.3.226 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_aug

## Save and puch results

In [ ]:
# Configure Git with your identity
!git config --global user.name {GITHUB_USER}
!git config --global user.email {GITHUB_EMAIL}

# Version the new model file
!dvc add models/best.pt

# Add the new .dvc file to Git
!git add models/best.pt.dvc .gitignore

# Commit the change
!git commit -m "Auto-train: Colab training run"

# Push the new commit to GitHub
!git push

# Push the new model file to Backblaze
!dvc push